In [63]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
import pandas as pd

/home/wklee/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [6]:
store = pd.HDFStore("../data/2016Aug_p.h5")
#store['df'] = df
df = store['df']
store.close()

In [7]:
for k in df:
    print(k)

title
abstract
categories
created
id
doi
label


In [211]:
abstracts = list(df['abstract'])
labels = np.array(df['label'])

# Tokenizer

In [212]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

In [213]:
tokenizer.fit_on_texts(abstracts)

In [214]:
sequences = tokenizer.texts_to_sequences(abstracts)

In [215]:
tokenizer.index_word

{1: 'the',
 2: 'of',
 3: 'and',
 4: 'a',
 5: 'in',
 6: 'to',
 7: 'we',
 8: 'with',
 9: 'is',
 10: 'that',
 11: 'for',
 12: 'are',
 13: 'from',
 14: 'by',
 15: 'on',
 16: 'at',
 17: 'this',
 18: '0',
 19: 'as',
 20: '1',
 21: 'be',
 22: 'an',
 23: 'mass',
 24: '2',
 25: 'which',
 26: 'our',
 27: 'these',
 28: 'star',
 29: 'data',
 30: 'stars',
 31: 'can',
 32: 'galaxies',
 33: 'have',
 34: 'model',
 35: '10',
 36: 'ray',
 37: 'observations',
 38: 'm',
 39: '3',
 40: 'high',
 41: 'emission',
 42: 'using',
 43: '5',
 44: 'find',
 45: 'between',
 46: 'stellar',
 47: 'x',
 48: 'than',
 49: 'also',
 50: 'or',
 51: 'field',
 52: 'it',
 53: 'two',
 54: 'observed',
 55: 'present',
 56: 'galaxy',
 57: 'large',
 58: 'energy',
 59: 'time',
 60: 'their',
 61: 'results',
 62: 'models',
 63: 'show',
 64: 'has',
 65: 'not',
 66: 'formation',
 67: '4',
 68: 'gas',
 69: 'z',
 70: 'density',
 71: 'low',
 72: 's',
 73: 'magnetic',
 74: 'h',
 75: 'its',
 76: 'rm',
 77: 'radio',
 78: 'based',
 79: 'dark',
 

In [216]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [483]:
seq = pad_sequences(sequences, padding='post', value=0, maxlen=100)

In [484]:
np.random.seed(1234)
ind = np.random.randint(0, len(labels), len(labels))
print(ind.shape)
labels = labels[ind]
seq = seq[ind,:]


split_1 = int(0.8 * len(labels))
split_2 = int(0.9 * len(labels))
train_labels = labels[:split_1]
dev_labels = labels[split_1:split_2]
test_labels = labels[split_2:]

train_seq = seq[:split_1, :]
dev_seq = seq[split_1:split_2, :]
test_seq = seq[split_2:, :]

(1806,)


In [485]:
vocab_size = 10000

In [486]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(6, activation=tf.nn.sigmoid))

In [487]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_9 ( (None, 16)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_19 (Dense)             (None, 6)                 102       
Total params: 160,374
Trainable params: 160,374
Non-trainable params: 0
_________________________________________________________________


In [488]:
from tensorflow.keras.optimizers import SGD

In [489]:
sgd = SGD(lr=0.01)

In [490]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [491]:
from tensorflow.keras.utils import to_categorical
train_labels_onehot = to_categorical(train_labels)

In [492]:
history = model.fit(train_seq, train_labels_onehot, epochs=20, steps_per_epoch=32, validation_split=0.3, validation_steps=32)

/home/wklee/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
32/32 [==============================] - 1s 46ms/step - loss: 1.7838 - acc: 0.2170 - val_loss: 1.7805 - val_acc: 0.1982
Epoch 2/20
32/32 [==============================] - 1s 20ms/step - loss: 1.7599 - acc: 0.2176 - val_loss: 1.7630 - val_acc: 0.1982
Epoch 3/20
32/32 [==============================] - 1s 21ms/step - loss: 1.7226 - acc: 0.2450 - val_loss: 1.7388 - val_acc: 0.2304
Epoch 4/20
32/32 [==============================] - 1s 21ms/step - loss: 1.6707 - acc: 0.3999 - val_loss: 1.7091 - val_acc: 0.3295
Epoch 5/20
32/32 [==============================] - 1s 20ms/step - loss: 1.5971 - acc: 0.4354 - val_loss: 1.6716 - val_acc: 0.3295
Epoch 6/20
32/32 [==============================] - 1s 21ms/step - loss: 1.4816 - acc: 0.4764 - val_loss: 1.6217 - val_acc: 0.3525
Epoch 7/20
32/32 [==============================] - 1s 21ms/step - loss: 1.3200 - acc: 0.5853 - val_loss: 1.5619 - val_acc: 0.3825
Epoch 8/20
32/32 [==============================] - 1s 21ms/step - loss: 1.1132 - a

In [493]:
from tensorflow.keras.utils import to_categorical

In [494]:
test_labels_onehot = to_categorical(test_labels)

In [495]:
model.evaluate(test_seq, test_labels_onehot, steps=10)

10/10 [==============================] - 0s 2ms/step


[1.7209223508834839, 0.49723756313323975]

In [504]:
text = ["we present high dispersion spectroscopic data of the compact planetary nebula vy 1 2 where high expansion velocities up to 100 km s are found in the ha n ii and o iii emission lines hst images reveal a bipolar structure vy 1 2 displays a bright ring like structure with a size of 2 4 2 and two faint bipolar lobes in the west east direction a faint pair of knots is also found located almost on opposite sides of the nebula at pa degrees furthermore deep low dispersion spectra are also presented and several emission lines are detected for the first time in this nebula such as the doublet cl iii a k iv a c ii 6461 a the doublet c iv 5801 5812 a by comparison with the solar abundances we find enhanced n depleted c and solar o the central star must have experienced the hot bottom burning cn cycle during the 2nd dredge up phase implying a progenitor star of higher than 3 solar masses the ver"]

In [505]:
seq_1 = tokenizer.texts_to_sequences(text)

In [506]:
seq_2 = pad_sequences(seq_1, padding='post', value=0, maxlen=350)

In [507]:
prob = model.predict(seq_2)
prob /= prob.sum()
print(prob)
ii = np.argmax(prob)
print(label2target[ii])

[[6.7621827e-01 3.0224794e-01 4.7230781e-03 5.4059108e-04 1.3514165e-02
  2.7558585e-03]]
astro-ph.GA


In [374]:
target_name_dict = {'astro-ph.GA' : 0,
                    'astro-ph.SR' : 1,
                    'astro-ph.IM' : 2,
                    'astro-ph.EP' : 3,
                    'astro-ph.HE' : 4,
                    'astro-ph.CO' : 5
                }
label2target = { v:k for k,v in target_name_dict.items()}
train_labels[0]

1

In [335]:
abstracts = list(abstracts)

In [441]:
j=100
print(label2target[train_labels[j]])
print(tokenizer.sequences_to_texts([train_seq[j]])[0])

astro-ph.SR
we present high dispersion spectroscopic data of the compact planetary nebula vy 1 2 where high expansion velocities up to 100 km s are found in the ha n ii and o iii emission lines hst images reveal a bipolar structure vy 1 2 displays a bright ring like structure with a size of 2 4 2 and two faint bipolar lobes in the west east direction a faint pair of knots is also found located almost on opposite sides of the nebula at pa degrees furthermore deep low dispersion spectra are also presented and several emission lines are detected for the first time in this nebula such as the doublet cl iii a k iv a c ii 6461 a the doublet c iv 5801 5812 a by comparison with the solar abundances we find enhanced n depleted c and solar o the central star must have experienced the hot bottom burning cn cycle during the 2nd dredge up phase implying a progenitor star of higher than 3 solar masses the very low c o and n o abundance ratios suggest a likely post common envelope close binary system

'astro-ph.EP'

In [247]:
abstracts[2]

"Low-mass star formation is described by gravitational collapse of dense cores\nof gas and dust. At some point during the collapse, a disk is formed around the\nprotostar and the disk will spin up and grow in size as the core contracts\nbecause of angular momentum conservation. The question is how early the disk\nformation process occurs. In this paper we aim to characterize the kinematical\nstate of a deeply embedded, Class 0 young stellar object, NGC1333-IRAS2A, based\non high angular resolution (< 1$''$ $\\approx$ 200 AU) interferometric\nobservations of HCN and H$^{13}$CN J = 4-3 from the Submillimeter Array, and\ntest whether a circumstellar disk can be detected based on gas kinematic\nfeatures. We adopt a physical model which has been shown to describe the object\nwell and obtain a fit of a parameterized model of the velocity field, using a\ntwo-dimensional axis-symmetric radiation transfer code. The parameterization\nand fit to the high angular resolution data characterize the c

In [265]:
test_seq.shape

(181, 350)

In [268]:
seq_2.shape

(1, 350)

In [327]:
?model.evaluate

In [328]:
model.metrics_names

['loss', 'acc']